https://www.reddit.com/r/LangChain/comments/13jd9wo/improving_the_quality_of_qa_with_pdf/

In [1]:
from dotenv import load_dotenv

In [2]:
if not load_dotenv():
    print("Cannot load .env file. Environment file is not exists or not readable")
    exit(1)

In [3]:
from langchain.document_loaders import PyPDFLoader

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document


In [5]:
import os
import glob
from typing import List

In [ ]:
# handle unstructured table

from langchain.document_loaders.parsers.pdf import PDFPlumberParser, PDFMinerParser

In [6]:
from langchain.document_loaders import (
    CSVLoader,
    EverNoteLoader,
    TextLoader,
    UnstructuredHTMLLoader,
    UnstructuredMarkdownLoader,
    UnstructuredODTLoader,
    UnstructuredPowerPointLoader,
    UnstructuredWordDocumentLoader,
    PyMuPDFLoader,
    PyPDFLoader
)

In [7]:
LOADER_MAPPING = {
    ".csv": (CSVLoader, {}),
    ".doc": (UnstructuredWordDocumentLoader, {}),
    ".docx": (UnstructuredWordDocumentLoader, {}),
    ".enex": (EverNoteLoader, {}),
    ".html": (UnstructuredHTMLLoader, {}),
    ".md": (UnstructuredMarkdownLoader, {}),
    ".odt": (UnstructuredODTLoader, {}),
    ".pdf": (PyPDFLoader, {}),
    ".ppt": (UnstructuredPowerPointLoader, {}),
    ".pptx": (UnstructuredPowerPointLoader, {}),
    ".txt": (TextLoader, {"encoding": "utf8"})
}

In [8]:
def load_single_document(file_path: str) -> List[Document]:
    ext = "." + file_path.rsplit(".", 1)[-1].lower()
    if ext in LOADER_MAPPING:
        loader_class, loader_args = LOADER_MAPPING[ext]
        loader = loader_class(file_path, **loader_args)
        return loader.load()
    
    # loader = PyPDFLoader(file_path)
    # return loader.load()

    raise ValueError(f"Unsupported file extension '{ext}'")

In [9]:
from multiprocessing import Pool
from tqdm import tqdm

In [10]:
def load_documents(source_dir: str, ignored_files: List[str] = []) -> List[Document]: 
    all_files = []
    for ext in LOADER_MAPPING:
        all_files.extend(
            glob.glob(os.path.join(source_dir, f"**/*{ext.lower()}"), recursive=True)
            )
        # all_files.extend(
        #     glob.glob(os.path.join(source_dir, f"**/*{ext.upper()}"), recursive=True)
        # )
    filtered_files = [file_path for file_path in all_files if file_path not in ignored_files]

    # multiprocessing
    # with Pool(processes=os.cpu_count()) as pool:
    #     results = []
    #     # progress bar
    #     with tqdm(total=len(filtered_files), desc='Loading new documents', ncols=80) as pbar:
    #         # apply function to each item in the iterable in parallel, pool.imap_unordered(task, items)
    #         for i, docs in enumerate(pool.imap_unordered(load_single_document, filtered_files)):
    #             results.extend(docs)
    #             pbar.update()

    results = []
    with tqdm(total=len(filtered_files), desc='Loading new documents', ncols=80) as pbar:
        for file_path in filtered_files:
            docs = load_single_document(file_path)
            results.extend(docs)
            pbar.update()

    return results

In [11]:
# Load environment variables
persist_directory = os.environ.get('PERSIST_DIRECTORY')
source_directory = os.environ.get('SOURCE_DIRECTORY', 'src_documents')
embeddings_model_name = os.environ.get('EMBEDDINGS_MODEL_NAME')
chunk_size = 500
chunk_overlap = 50

In [12]:
# from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("src_documents\\DIALOGPT.pdf")
pages = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n"],
    chunk_size=chunk_size, 
    chunk_overlap=chunk_overlap,
    length_function=len)

docs = text_splitter.split_documents(pages)

len(docs)
# len(pages)

89

In [13]:
if __name__ == "__main__":
    documents = load_documents(source_directory, [])
    print(len(documents))

Loading new documents: 100%|██████████████████████| 1/1 [00:00<00:00,  5.36it/s]

10


In [14]:
def process_documents(ignored_files: List[str] = []) -> List[Document]:
    """
    Load documents and split in chunks
    """
    print(f"Loading documents from {source_directory}")
    documents = load_documents(source_directory, ignored_files)
    if not documents:
        print("No new documents to load")
        exit(0)
    print(f"Loaded {len(documents)} new documents from {source_directory}")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    texts = text_splitter.split_documents(documents)
    print(f"Split into {len(texts)} chunks of text (max. {chunk_size} tokens each)")
    return texts

In [15]:
texts = process_documents()

Loading documents from src_documents


Loading new documents: 100%|██████████████████████| 1/1 [00:00<00:00,  4.85it/s]

Loaded 10 new documents from src_documents
Split into 89 chunks of text (max. 500 tokens each)


In [16]:
def does_vectorstore_exist(persist_directory: str, embeddings: HuggingFaceEmbeddings, settings) -> bool:
    """
    Checks if vectorstore exists
    """
    db = Chroma(persist_directory=persist_directory, embedding_function=embeddings, client_settings=settings)
    if not db.get()['documents']:
        return False
    return True

In [17]:
from constants import CHROMA_SETTINGS
import chromadb

In [19]:
def main():
    # Create embeddings
    embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)
    # Chroma client
    chroma_client = chromadb.PersistentClient(settings=CHROMA_SETTINGS , path=persist_directory)

    if does_vectorstore_exist(persist_directory, embeddings, CHROMA_SETTINGS):
        # Update and store locally vectorstore
        print(f"Appending to existing vectorstore at {persist_directory}")
        db = Chroma(persist_directory=persist_directory, embedding_function=embeddings, client_settings=CHROMA_SETTINGS, client=chroma_client)
        collection = db.get()
        print(collection)
        texts = process_documents([metadata['source'] for metadata in collection['metadatas']])
        print(f"Creating embeddings. May take some minutes...")
        db.add_documents(texts)
        print(True)
    else:
        # Create and store locally vectorstore
        print("Creating new vectorstore")
        texts = process_documents()
        print(f"Creating embeddings. May take some minutes...")
        db = Chroma.from_documents(texts, embeddings, persist_directory=persist_directory, client_settings=CHROMA_SETTINGS, client=chroma_client)
        print(False)
    db.persist()
    db = None

    print(f"Ingestion complete! You can now run privateGPT.py to query your documents")

In [20]:
main()

Creating new vectorstore
Loading documents from src_documents


Loading new documents: 100%|██████████████████████| 1/1 [00:00<00:00,  4.13it/s]


Loaded 10 new documents from src_documents
Split into 89 chunks of text (max. 500 tokens each)
Creating embeddings. May take some minutes...
False
Ingestion complete! You can now run privateGPT.py to query your documents
